In [1]:
library(scMetabolism)
library(ggplot2)
library(rsvd)
library(pheatmap)

### read input files and calculate scores 

In [2]:
hcc <- readRDS("HCC02_B_bin50_tissue_only_spotlight_Spatial.rds")
hcc@assays$RNA <- hcc@assays$Spatial
hcc <- sc.metabolism.Seurat(obj = hcc, method = "VISION", imputation = F, ncores = 2, metabolism.type = "KEGG")
input.pathway <- rownames(hcc@assays$METABOLISM$score)

载入需要的程辑包：SeuratObject

载入需要的程辑包：sp

The legacy packages maptools, rgdal, and rgeos, underpinning the sp package,
which was just loaded, will retire in October 2023.
Please refer to R-spatial evolution reports for details, especially
https://r-spatial.org/r/2023/05/15/evolution4.html.
It may be desirable to make the sf package available;
package maintainers should consider adding sf to Suggests:.
The sp package is now running under evolution status 2
     (status 2 uses the sf package in place of rgdal)



Your choice is: KEGG
Start quantify the metabolism activity...


Loading data from /home/fangqi/bin/R-4.2.1/lib/R/library/scMetabolism/data/KEGG_metabolism_nc.gmt ...

Warning message in readSignaturesInput(signatures):
“强制改变过程中产生了NA”

Using 21107/31831 genes detected in 0.10% of cells for signature analysis.

See the `sig_gene_threshold` input to change this behavior.


Beginning Analysis


Computing a latent space for expression data...


Determining projection genes...

    Applying Threshold filter...removing genes detected in less than 100 cells

      Genes Retained: 5306

    Applying Fano filter...removing genes with Fano < 2.0 MAD in each of 30 bins

      Genes Retained: 715



Clustering cells...

Using latent space to cluster cells...

completed


Projecting data into 2 dimensions...

  Running method 1/1: tSNE30 ...



Evaluating signature scores on cells...


as(<matrix>, "dgeMatrix") is deprecated since Matrix 1.5-0; do as(as(as(., "dMatrix"), "generalMatrix"), "unpackedMatrix") instead

Evaluating signature-gene importance...


Creat

  sigSize sigBalance
1       6          1
2      12          1
3      20          1
4      33          1
5      63          1


  signatures per group: 3000

Computing KNN Cell Graph in the Latent Space...


Evaluating local consistency of signatures in latent space...


Clustering signatures...


Computing differential signature tests...


Computing correlations between signatures and latent space components...


Analysis Complete!




### plot results

In [3]:
p1 <- DotPlot.metabolism(obj=hcc,pathway=input.pathway,phenotype="SpotLight_Anno",norm="y")
pdf("HCC02_B_scmetabolism_dotplot.pdf",width=10,height=15)
print(p1)
dev.off()

png 
  2

In [4]:
sce_Metal_exp <- hcc
sce_Metal_exp$celltype <- sce_Metal_exp$SpotLight_Anno
mscore_data <- data.frame(t(sce_Metal_exp@assays[["METABOLISM"]][["score"]]),sce_Metal_exp$celltype)
avg_sM <- aggregate(mscore_data[,1:ncol(mscore_data)-1],list(mscore_data$sce_Metal_exp.celltype),mean)
rownames(avg_sM) = avg_sM$Group.1
avg_sM <- data.frame(t(avg_sM[,-1]))
avg_sM$KEGG = rownames(sce_Metal_exp@assays[["METABOLISM"]][["score"]])
rownames(avg_sM) <- avg_sM$KEGG

pheatmap(avg_sM[,-ncol(avg_sM)],
         show_colnames = T,
         scale='row',
         cellwidth = 15, 
         cellheight = 10, 
         fontsize = 6, 
         filename = "HCC02_B_scmetabolism_avg_sM.pdf")

载入需要的程辑包：Seurat



In [5]:
c_k_l <- c()
for(c in c(1:ncol(avg_sM))){
  c_k <- avg_sM[order(avg_sM[,c]),]$KEGG[1:5]
  c_k_l <- c(c_k_l,c_k)
}
c_k_l= unique(c_k_l)
c_k_d = avg_sM[avg_sM$KEGG %in%c_k_l,]
rownames(c_k_d) = c_k_d$KEGG

pheatmap(c_k_d[,-ncol(c_k_d)], 
         show_colnames = T, 
         scale='row',
         cellwidth = 15, 
         cellheight = 10, 
         fontsize = 6, 
         filename = "HCC02_B_kegg_ckd.pdf")

In [6]:
write.csv(hcc@assays[["METABOLISM"]][["score"]],"HCC02_B_bin50.scMetabolism.allBins.csv")
write.csv(avg_sM,"HCC02_B_bin50.scMetabolism.allCelltypes.csv")